In [27]:
import pandas as pd
import numpy as np

In [28]:
def data_loader(file_path):
    # Read the data from the text file
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.readlines()

    # Initialize lists to store data
    sent_ids = []
    texts = []
    tokens = []
    POS_tags = set() # POS Tag list
    dep_rels = set() # Dependency list

    # Process the data and extract information
    for line in data:
        if line.startswith('# sent_id'):
            sent_id = line.split('=')[1].strip()
            sent_ids.append(sent_id)
            tokens_list = []
        elif line.startswith('# text'):
            text = line.split('=')[1].strip()
            texts.append(text)
        elif line.strip() and not line.startswith('#'):
            parts = line.strip().split()
            POS_tags.add(parts[3]) # add the POS_tags
            dep_rels.add(parts[5]) # add dependencies
            tokens_list.append(' '.join(parts))

        # If an empty line is encountered, add tokens to the list and reset the list
        elif line.strip() == '' and len(tokens_list) > 0:
            tokens.append('|'.join(tokens_list))

    if len(tokens_list) > 0:
        tokens.append('|'.join(tokens_list))


    # Create dataframes
    df = pd.DataFrame({'sent_id': sent_ids, 'text': texts, 'token': tokens})

    return df, POS_tags, dep_rels

def get_vocab(df):
    vocab_count = {}
    for idx, row in df.iterrows():
        vocab = set()
        for token in row['token'].split('|'):
            parts = token.split(' ')
            vocab.add(parts[2])

        for word in vocab:
            if word in vocab_count:
                vocab_count[word] += 1
            else:
                vocab_count[word] = 1

    # print(vocab_count['appreciation'])
    vocab_count = {k: v for k, v in sorted(vocab_count.items(), key=lambda item: item[1], reverse=True)}

    num_sent = df.shape[0]
    vocab = []
    for word, count in vocab_count.items():
        if count <= num_sent/2:
            vocab.append(word)
        if len(vocab) == 1000:
            break
    
    return vocab

In [29]:
# Specify the path to the text file
train_path = 'NLP2/train.txt'
test_path = 'NLP2/test.txt'

df_train, POS_tags, dep_rels = data_loader(train_path)
df_test, _, _ = data_loader(test_path)

df_train.head()

,sent_id,text,token
0,GUM_academic_art-1,Aesthetic Appreciation and Spanish Art:,1 Aesthetic aesthetic JJ 2 amod|2 Appreciation...
1,GUM_academic_art-2,Insights from Eye-Tracking,1 Insights insight NNS 0 root|2 from from IN 5...
2,GUM_academic_art-3,Claire Bailey-Ross claire.bailey-ross@port.ac....,1 Claire Claire NNP 0 root|2 Bailey Bailey NNP...
3,GUM_academic_art-4,Andrew Beresford a.m.beresford@durham.ac.uk Du...,1 Andrew Andrew NNP 0 root|2 Beresford Beresfo...
4,GUM_academic_art-5,Daniel Smith daniel.smith2@durham.ac.uk Durham...,1 Daniel Daniel NNP 0 root|2 Smith Smith NNP 1...


In [30]:
vocab = get_vocab(df_train)
print(f'Vocabulary Length : {len(vocab)}')

Vocabulary Length : 1000


In [31]:
POS_map = {}
DEP_map = {}
VOCAB_map = {}

for i, POS in enumerate(POS_tags):
    POS_map[POS] = i

for i, DEP in enumerate(dep_rels):
    DEP_map[DEP] = i

for i, VOC in enumerate(vocab):
    VOCAB_map[VOC] = i


In [32]:
class ParserConfiguration:
    def __init__(self, sentence):
        self.org = sentence[:]
        self.stack = []  # Stack initially empty
        self.buffer = sentence[:]  # Buffer contains all tokens (shallow copy)
        self.arcs = []  # List of tuples for arcs; each tuple is (head, dependent)

    def left_arc(self, dep_rel):
        if len(self.stack) > 0:
            # s = self.stack.pop()  # Remove the top of the stack
            s = self.stack[-1]  # Look at the top of the stack
            b = self.buffer[0]  # Look at the first item in the buffer
            
            head_assigned = False # Check if head for first(B)
            for arc in self.arcs: # Check if top(S) have a head word
                if arc[1] == s[0]:
                    head_assigned = True
                    break
            
            if not head_assigned:
                s = self.stack.pop()  # Remove the top of the stack
                self.arcs.append((b[0], s[0], dep_rel))  # Add arc B->S with dependency relation
                return 1
            else:
                return 0
        
        return -1

    def right_arc(self, dep_rel):
        if len(self.buffer) > 0:
            s = self.stack[-1]  # Look at the top of the stack
            b = self.buffer.pop(0)  # Remove the first item from the buffer
            self.stack.append(b)  # Push B onto the stack
            self.arcs.append((s[0], b[0], dep_rel))  # Add arc S->B with dependency relation

    def reduce(self):
        if len(self.stack) > 0:
            s = self.stack[-1]  # Look at the top of the stack
            head_assigned = False
            for arc in self.arcs: # Check if top(S) have a head word
                if arc[1] == s[0]:
                    head_assigned = True
                    break

            if head_assigned:
                self.stack.pop()  # Remove the top of the stack
                return 1
            else:
                return 0
            
        return -1

    def shift(self):
        if len(self.buffer) > 0:
            b = self.buffer.pop(0)  # Remove the first item from the buffer
            self.stack.append(b)  # Push it onto the stack

    def display_state(self):
        print(f'\nOriginal : {self.org}')
        print(f'Stack : {self.stack}')
        print(f'Buffer : {self.buffer}')
        print(f'Arcs : {self.arcs}')


In [33]:
class FeatureVector:
    def __init__(self, config, VOCAB_map, POS_map, DEP_map):
        self.V_size = len(VOCAB_map)
        self.P_size = len(POS_map)
        self.R_size = len(DEP_map)
        self.feature_vector_size = 4 * (2*self.V_size + 3*self.P_size + 4*self.R_size)
        self.f_vec = np.zeros(self.feature_vector_size, dtype=int)
        self.S, self.B, self.A = config.stack, config.buffer, config.arcs
        self.conf = config
        self.VOCAB_map = VOCAB_map
        self.POS_map = POS_map
        self.DEP_map = DEP_map

    # Helper function to update feature vector for a given feature index
    def set_feature(self, idx):
        if 0 <= idx < self.feature_vector_size:
            self.f_vec[idx] = 1
    
    def update_top_S_DEP(self, top_S, offset):
        head_top_S = top_S[3]
        left_most_w = self.conf.org[0]
        right_most_w = self.conf.org[-1]

        for (head, dep, rel) in self.A:
            if rel != None:
                if head == head_top_S and dep == top_S[0]:
                    self.set_feature(self.V_size + self.P_size + self.DEP_map[rel] + offset)
                elif head == top_S[0] and dep == left_most_w[0]:
                    self.set_feature(self.V_size + self.P_size + self.R_size + self.DEP_map[rel] + offset)
                elif head == top_S[0] and dep == right_most_w[0]:
                    self.set_feature(self.V_size + self.P_size + 2*self.R_size + self.DEP_map[rel] + offset)

    def update_buffer_DEP(self,first_B, offset):
        left_most_w = self.conf.org[0]
        for (head, dep, rel) in self.A:
            if rel != None:
                if head == first_B[0] and dep == left_most_w[0]:
                    self.set_feature(2*self.V_size + 2*self.P_size + 3*self.R_size + self.DEP_map[rel] + offset)


    def get_feature_vector(self, transition):   
        # Feature indices based on the transition
        transition_map = {'LEFT-ARC': 0, 'RIGHT-ARC': 1, 'REDUCE': 2, 'SHIFT': 3}
        transition_offset = transition_map[transition] * (2*self.V_size + 3*self.P_size + 4*self.R_size)
        
        # TOP feature
        if self.S:
            top_S = self.S[-1]  # Last stack item is the top
            # print(top_S)

            # TOP
            top_token = top_S[1]  # Normalized token
            if top_token in self.VOCAB_map:
                self.set_feature(self.VOCAB_map[top_token] + transition_offset)
            
            # TOP.POS
            top_POS = top_S[2] # POS_tag for TOS
            if top_POS in self.POS_map:
                self.set_feature(self.V_size + self.POS_map[top_POS] + transition_offset)
            
            # Update DEP features : TOP.DEP, TOP.RDEP, TOP.LDEP
            self.update_top_S_DEP(top_S, transition_offset)
        
        # FIRST feature
        if self.B:
            first_B = self.B[0]  # First buffer item 
            
            # FIRST
            first_token = first_B[1] # Normalized token
            if first_token in self.VOCAB_map:
                self.set_feature(self.V_size + self.P_size + 3 * self.R_size + self.VOCAB_map[first_token] + transition_offset)
            
            # FIRST.POS
            first_POS = first_B[2] # POS_tag for first(Buffer)
            if first_POS in self.POS_map:
                self.set_feature(2 * self.V_size + self.P_size + 3 * self.R_size + self.POS_map[first_POS] + transition_offset)
            
            # Update DEP features : FIRST.LDEP
            self.update_buffer_DEP(first_B, transition_offset)
        
        # LOOK.POS
        if len(self.B) >= 2:
            second_B = self.B[1]
            second_POS = second_B[2]
            self.set_feature(2 * self.V_size + 2 * self.P_size + 4 * self.R_size + self.POS_map[second_POS] + transition_offset)
        
        return self.f_vec

        

In [34]:
def check_REDUCE(stack, buffer, top_of_stack , gold_arcs, existing_arcs):
    '''
    stack = list[]
    buffer = list[]
    top_of_stack : int (index of TOS)
    gold_arcs : dict{}
    existing_arcs : dict{}
    '''
    exists_dep = False
    for arcs in existing_arcs:
        if top_of_stack == arcs[1]: # head of top_of_stack has been assigned
            for element in stack: # check for a 'w' in the stack
                ele_id = element[0]
                if ele_id != top_of_stack: # w != top(S)
                    if (ele_id, buffer[0][0]) in gold_arcs or (buffer[0][0], ele_id) in gold_arcs:
                        exists_dep = True
                        break
    
    return exists_dep

def oracle(conf, gold_arcs):
    S, B, A = conf.stack, conf.buffer, conf.arcs
    
    if not S:
        return 'SHIFT', None  # If stack is empty, we can only SHIFT.
    
    top_of_stack = S[-1][0]  # Assuming each item in S and B is [token_id, ...]
    first_of_buffer = B[0][0] if B else None

    # Convert set of arcs A to a more searchable structure
    existing_arcs = {(head, dep) for head, dep, rel in A}

    # Rule 1: LEFT-ARC
    if first_of_buffer and (first_of_buffer, top_of_stack) in gold_arcs: # If first(B) -> top(S) exists in the dependency graph
        head_assigned = False
        for arcs in existing_arcs: # Check if top(S) already has a head
            if top_of_stack == arcs[1]:
                head_assigned = True
                break
        if not head_assigned:
            return 'LEFT-ARC', gold_arcs[(first_of_buffer, top_of_stack)]
    
    # Rule 2: RIGHT-ARC
    if first_of_buffer and (top_of_stack, first_of_buffer) in gold_arcs:
        return 'RIGHT-ARC', gold_arcs[(top_of_stack, first_of_buffer)]
    
    # Rule 3: REDUCE
    if check_REDUCE(S, B, top_of_stack, gold_arcs, existing_arcs):
        return 'REDUCE', None
    
    # Default to SHIFT if none of the other conditions are met.
    return 'SHIFT', None


In [35]:
def generate_training_instances(sentences, oracle):
    training_instances = []
    
    for sentence in sentences:
        # Initial configuration for the sentence
        conf = ParserConfiguration(sentence)
        
        # Extract gold-standard arcs from the sentence
        gold_arcs = {(word[3], word[0]): word[4] for word in sentence}

        while len(conf.buffer) >= 1:  # Continue until buffer is empty 
            gold_action, dep_rel = oracle(conf, gold_arcs)
            # Encode current configuration and gold action into feature vector
            training_instances.append((conf, gold_action))
            
            # Update configuration based on the gold action
            if gold_action == 'LEFT-ARC':
                conf.left_arc(dep_rel)
            elif gold_action == 'RIGHT-ARC':
                conf.right_arc(dep_rel)
            elif gold_action == 'REDUCE':
                conf.reduce()
            elif gold_action == 'SHIFT':
                conf.shift()
        
    return training_instances



In [36]:
def train_classifier(training_data, num_features, num_epochs, VOCAB_map, POS_map, DEP_map):
    """
    Trains a linear classifier using the Perceptron algorithm.
    
    training_data: A list of tuples (feature_vector, gold_action) for training.
    num_features: The number of features in each feature vector.
    num_epochs: The number of passes over the training data.
    A NumPy array representing the trained weights.
    """
    actions = ['LEFT-ARC', 'RIGHT-ARC', 'REDUCE', 'SHIFT']

    index_to_action = {i: action for i, action in enumerate(actions)}
    W = np.random.rand(num_features)

    
    for epoch in range(num_epochs):
        for conf, gold_action in training_data:
            # Predict action
            scores = []
            for test_action in actions:
                fv = FeatureVector(conf, VOCAB_map, POS_map, DEP_map)
                f_vec = fv.get_feature_vector(test_action)
                score = np.dot(W, f_vec)
                scores.append(score)

            predicted_action_index = np.argmax(scores)
            predicted_action = index_to_action[predicted_action_index]
            
            # Update weights if prediction is wrong
            if predicted_action != gold_action:
                feature_vector_pred = fv.get_feature_vector(predicted_action)
                feature_vector_gold = fv.get_feature_vector(gold_action)
                W = W + feature_vector_gold - feature_vector_pred

        print(f'Epoch : {epoch + 1}')
                
    return W



In [37]:
def predict_action(sentences, weights, POS_DEP_map):
    actions = ['LEFT-ARC', 'RIGHT-ARC', 'REDUCE', 'SHIFT']
    index_to_action = {i: action for i, action in enumerate(actions)}
    arc_list = []

    for sentence in sentences:
        # print('Parsing', count)
        conf = ParserConfiguration(sentence)
        while len(conf.buffer) >= 1:
            scores = []
            for test_action in actions:
                fv = FeatureVector(conf, VOCAB_map, POS_map, DEP_map)
                f_vec = fv.get_feature_vector(test_action)
                score = np.dot(weights, f_vec)
                scores.append(score)
            
            predicted_action_index = np.argmax(scores)
            predicted_action = index_to_action[predicted_action_index]
            
            if predicted_action == 'LEFT-ARC':
                top_S_POS = None
                first_B_POS = None
                dep = None

                if len(conf.stack) > 0:
                    top_S_POS = conf.stack[-1][2]
                first_B_POS = conf.buffer[0][2]

                if (first_B_POS, top_S_POS) in POS_DEP_map:
                    dep = POS_DEP_map[(first_B_POS, top_S_POS)]

                flag = conf.left_arc(dep)
                if flag == -1 : # Empty stack heuristic
                    conf.shift()
                if flag == 0: # Head already assigned Heuristic
                    conf.reduce()

            elif predicted_action == 'RIGHT-ARC':
                if len(conf.stack) > 0:
                    top_S_POS = conf.stack[-1][2]
                    first_B_POS = conf.buffer[0][2]
                    
                    if (top_S_POS, first_B_POS) in POS_DEP_map:
                        dep = POS_DEP_map[(top_S_POS, first_B_POS)]

                    conf.right_arc(dep)
                else:
                    conf.shift() # Heuristic

            elif predicted_action == 'REDUCE':
                flag = conf.reduce()

                if flag == -1: # Empty stack heuristic
                    conf.shift()
                if flag == 0: # Head not assigned Heuristic
                    top_S_POS = conf.stack[-1][2]
                    first_B_POS = conf.buffer[0][2]

                    if (first_B_POS, top_S_POS) in POS_DEP_map:
                        dep = POS_DEP_map[(first_B_POS, top_S_POS)]

                    conf.left_arc(dep)

            elif predicted_action == 'SHIFT':
                conf.shift()
        

        arc_list.append([(arc[0], arc[1]) for arc in conf.arcs])
        # arc_list.append(conf.arcs)

    return arc_list

def calculate_UAS(arc_list, gold_arcs):
    scores = []
    for idx, sent in enumerate(gold_arcs):
        correct = 0
        for arc in sent:
            if arc in arc_list[idx]:
                correct += 1
        scores.append(correct/len(gold_arcs[idx]))

    return sum(scores)/len(scores)


In [38]:
def get_POS_approximation(sentences):
    POS_pair_DEP = {}
    for sentence in sentences:
        id_to_POS = {}
        for token in sentence:
            id_to_POS[token[0]] = token[2]

        for token in sentence:
            dep = token[0]
            head = token[3]
            rel = token[4]
            if head != '0' and head != '_':
                if (id_to_POS[head], id_to_POS[dep]) in POS_pair_DEP:
                    DEP_list = POS_pair_DEP[(id_to_POS[head], id_to_POS[dep])]
                    
                    if rel in DEP_list:
                        POS_pair_DEP[(id_to_POS[head], id_to_POS[dep])][rel] += 1
                    else:
                        POS_pair_DEP[(id_to_POS[head], id_to_POS[dep])][rel] = 1

                else:
                    POS_pair_DEP[(id_to_POS[head], id_to_POS[dep])] = {}
                    POS_pair_DEP[(id_to_POS[head], id_to_POS[dep])][rel] = 1
    
    POS_DEP_map = {}
    for POS_pair, DEP_dict in POS_pair_DEP.items():
        dep = max(DEP_dict, key=DEP_dict.get)
        POS_DEP_map[POS_pair] = dep

    return POS_DEP_map

In [39]:
def get_true_arcs(sentences):
    true_arcs_list = []
    for sentence in sentences:
        true_arcs_list.append([(word[3], word[0]) for word in sentence])

    return true_arcs_list

In [40]:
# Prepare the sentences for the train and test

train_sentences = []
for idx, row in df_train.iterrows():
    sentence = []
    tok_list = row['token'].split('|')
    for tok in tok_list:
        parts = tok.split()
        final_token = [parts[0], parts[2], parts[3], parts[4], parts[5]]
        sentence.append(final_token)
    train_sentences.append(sentence)

test_sentences = []
for idx, row in df_test.iterrows():
    sentence = []
    tok_list = row['token'].split('|')
    for tok in tok_list:
        parts = tok.split()
        final_token = [parts[0], parts[2], parts[3], parts[4], parts[5]]
        sentence.append(final_token)
    test_sentences.append(sentence)

In [41]:
training_instances  = generate_training_instances(train_sentences, oracle)

In [42]:
num_features = 4 * (2 * len(VOCAB_map) + 3 * len(POS_map) + 4 * len(DEP_map))
weights = train_classifier(training_instances, num_features, 50, VOCAB_map, POS_map, DEP_map)
np.save('dependency_predictions_on.npy', weights)

Epoch : 1
Epoch : 2
Epoch : 3
Epoch : 4
Epoch : 5
Epoch : 6
Epoch : 7
Epoch : 8
Epoch : 9
Epoch : 10
Epoch : 11
Epoch : 12
Epoch : 13
Epoch : 14
Epoch : 15
Epoch : 16
Epoch : 17
Epoch : 18
Epoch : 19
Epoch : 20
Epoch : 21
Epoch : 22
Epoch : 23
Epoch : 24
Epoch : 25
Epoch : 26
Epoch : 27
Epoch : 28
Epoch : 29
Epoch : 30
Epoch : 31
Epoch : 32
Epoch : 33
Epoch : 34
Epoch : 35
Epoch : 36
Epoch : 37
Epoch : 38
Epoch : 39
Epoch : 40
Epoch : 41
Epoch : 42
Epoch : 43
Epoch : 44
Epoch : 45
Epoch : 46
Epoch : 47
Epoch : 48
Epoch : 49
Epoch : 50


In [24]:
POS_DEP_map = get_POS_approximation(train_sentences)

train_pred_arcs = predict_action(train_sentences, weights, POS_DEP_map)
train_true_arcs = get_true_arcs(train_sentences)
UAS_train = calculate_UAS(train_pred_arcs, train_true_arcs)
print(f'UAS Train Set: {UAS_train}')

test_pred_arcs = predict_action(test_sentences, weights, POS_DEP_map)
test_true_arcs = get_true_arcs(test_sentences)
UAS_test = calculate_UAS(test_pred_arcs, test_true_arcs)
print(f'UAS Test Set: {UAS_test}')

UAS Train Set: 0.14903587485786365
UAS Test Set: 0.1478183209857962


In [44]:
import csv

with open('dependency_predictions_on.tsv', mode='w+', newline='', encoding="utf-8") as file:    
    writer = csv.writer(file, delimiter='\t', lineterminator='\n')
    for idx, arc_list in enumerate(test_pred_arcs):
        for arc in arc_list:
            head, dep = arc
            
            for tok in test_sentences[idx]:
                if tok[0] == dep:
                    writer.writerow([df_test.iloc[idx]["sent_id"], dep, tok[1], head])
                    break